In [1]:
from pyathena import connect
from pyathena.util import as_pandas

In [2]:
import pandas as pd

In [3]:
products = pd.read_csv('data/products.csv', dtype={'gtin':'str'})

In [4]:
prods = tuple(products.gtin)

In [14]:
def get_data(products):
    """
    Queries Athena and gets daily GMV
    """
    cursor = connect(aws_access_key_id='',
                     aws_secret_access_key='',
                     s3_staging_dir='',
                     region_name='').cursor()

    cursor.execute("""
SELECT availability_days, brand, branded_store_slug, canonical_sku, catalog_feed_date, catalog_feed_id, category, category_info, channel_slug, commission_plan, created_at, currency, description, external_id, group_id, gtin, id, name, offer, offer_discount, parent_id, part_number, partition_0, price, price_freight_shift, reject_reason, seller_product_sku, sent_error_reason, status, stock, updated_at
FROM "olist-datalake-athena".channels_api_products_channelproducthistory
where gtin in {}
UNION
SELECT availability_days, brand, branded_store_slug, canonical_sku, catalog_feed_date, catalog_feed_id, category, category_info, channel_slug, commission_plan, created_at, currency, description, external_id, group_id, gtin, id, name, offer, offer_discount, parent_id, part_number, partition_0, price, price_freight_shift, reject_reason, seller_product_sku, sent_error_reason, status, stock, updated_at
FROM "olist-datalake-athena".channels_api_products_channelproducthistory_2019
where gtin in {}
UNION
SELECT availability_days, brand, branded_store_slug, canonical_sku, catalog_feed_date, catalog_feed_id, category, category_info, channel_slug, commission_plan, created_at, currency, description, external_id, group_id, gtin, id, name, offer, offer_discount, parent_id, part_number, partition_0, price, price_freight_shift, reject_reason, seller_product_sku, sent_error_reason, status, stock, updated_at
FROM "olist-datalake-athena".channels_api_products_channelproducthistory_2020
where gtin in {}
""".format(products,products,products))

    df = as_pandas(cursor)


    return df

In [15]:
df = get_data(prods)

In [18]:
df.to_csv('data/full_history.csv', index=False)

In [9]:
d = pd.read_csv('data/full_history.csv')

/usr/local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (0,5,15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [11]:
d[d['gtin']=='7898164712364']

,availability_days,brand,branded_store_slug,canonical_sku,catalog_feed_date,catalog_feed_id,category,category_info,channel_slug,commission_plan,...,part_number,partition_0,price,price_freight_shift,reject_reason,seller_product_sku,sent_error_reason,status,stock,updated_at
32875,0,Grill Oriental,olist,PRDQE46ME2FLUENA,NaN,NaN,16a2058f-8065-4319-baa6-0ce468ff9554,NaN,saraiva,default,...,NaN,10.0,359.0,0.0,NaN,PRD00IZ9G084MDHK,NaN,ready,5.0,2019-10-28 14:50:48
34173,2,Grill Oriental,olist,PRDRN5MVIBXZAGL2,NaN,NaN,c3aea8e6-bcea-4ee6-ad01-6b7012fe4924,"{""id"": ""c3aea8e6-bcea-4ee6-ad01-6b7012fe4924"",...",amazon,default,...,NaN,1.0,359.0,0.0,NaN,PRD00IZ9G084MDHK,NaN,published,90.0,2020-01-14 18:16:02
36168,0,Grill Oriental,olist,PRDDA8SFRXPIUYWQ,NaN,NaN,16a2058f-8065-4319-baa6-0ce468ff9554,NaN,zoom,default,...,NaN,10.0,359.0,0.0,NaN,PRD00IZ9G084MDHK,NaN,ready,5.0,2019-10-28 14:51:00
40713,0,Grill Oriental,olist,PRDQE46ME2FLUENA,NaN,NaN,16a2058f-8065-4319-baa6-0ce468ff9554,NaN,saraiva,default,...,NaN,10.0,359.0,0.0,NaN,PRD00IZ9G084MDHK,NaN,pending,5.0,2019-10-28 14:49:58
43395,0,Grill Oriental,olist,PRD4M6UTYV2P6JWU,NaN,NaN,16a2058f-8065-4319-baa6-0ce468ff9554,NaN,madeiramadeira,default,...,NaN,10.0,359.0,0.0,NaN,PRD00IZ9G084MDHK,NaN,pending,5.0,2019-10-28 14:49:56
46956,0,Grill Oriental,olist,PRDNOB5JXO226YHY,NaN,NaN,255283ab-30c6-46a9-af8f-3c43e4508b3b,"{""id"": ""255283ab-30c6-46a9-af8f-3c43e4508b3b"",...",mercadolivre,reduced,...,NaN,10.0,359.0,0.0,NaN,PRD00IZ9G084MDHK,NaN,sent,5.0,2019-10-28 14:50:25
50506,2,Grill Oriental,olist,PRDRN5MVIBXZAGL2,NaN,NaN,c3aea8e6-bcea-4ee6-ad01-6b7012fe4924,"{""id"": ""c3aea8e6-bcea-4ee6-ad01-6b7012fe4924"",...",amazon,default,...,NaN,11.0,359.0,0.0,NaN,PRD00IZ9G084MDHK,NaN,published,29.0,2019-11-29 03:32:33
50541,0,Grill Oriental,olist,PRD2VNIWZ5RGYVJU,NaN,NaN,255283ab-30c6-46a9-af8f-3c43e4508b3b,"{""id"": ""255283ab-30c6-46a9-af8f-3c43e4508b3b"",...",mercadolivre,default,...,NaN,11.0,359.0,0.0,NaN,PRD00IZ9G084MDHK,NaN,published,29.0,2019-11-29 18:41:14
